In [1]:
import numpy as np
import pandas as pd
import requests
import datetime
import time
import json

In [2]:
# all stocks from Fidelity website as of 2022
stock_list = pd.read_csv('screener_results.csv')

In [3]:
stock_list = stock_list[['Symbol', 'Sector', 'Industry']]

In [4]:
from secrets import special_api_key
import functions as f

In [19]:
collect_data(776)

ConnectionError: HTTPSConnectionPool(host='www.alphavantage.co', port=443): Max retries exceeded with url: /query?function=INCOME_STATEMENT&symbol=RE&apikey=XFQRBMDPUZ9G0CIA (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f4d9d718b50>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

In [8]:
def collect_data(start=0, stop=None):

    x = start

    for stock, sector, industry in stock_list.values[start:stop]:

        income_data, cash_data, balance_data = f.make_api_call_for_financial_statements(stock, special_api_key)
        price_data = f.make_api_call_for_price_time_series(stock, special_api_key)
        
        if len(income_data) == 0 or len(cash_data) == 0 or len(balance_data) == 0: 
            
            print(f'Error: {stock}')
            
            with open(f'./error-lists/collect-load-error.csv', 'a') as file: 
                file.write(f'{stock}\n')
            
            x += 1
            continue
        
        master_dict = {'ticker': stock, 'sector': sector, 'industry': industry, 'income_data': income_data, 'cash_data': cash_data, 'balance_data': balance_data, 'price_data': price_data}
        
        time.sleep(61)
        
        print(x, stock)
        x += 1

        with open(f'json_files/{stock}.json', 'w') as file: 
            file.write(json.dumps(master_dict))